In [24]:
import polars as pl
import pandas as pd
import time
import os

In [25]:
data_oct = '../data/2019-Oct.csv'
data_nov = '../data/2019-Nov.csv'
timing_path = '../stats/timings.csv' 

In [46]:
if os.path.exists(timing_path):
    timing = pd.read_csv(timing_path, index_col=0)
else:
    timing = pd.DataFrame(columns=['Timing_Pandas', 'Timing_Polars'])

1. Lecture des deux jeux de données et concatenation dans un seul dataset

In [27]:
ex = 1

start_time = time.time()

data_raw_oct = pl.read_csv(data_oct)
data_raw_nov = pl.read_csv(data_nov)

data = pl.concat([data_raw_oct, data_raw_nov])

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

Temps d'exécution de l'exercise 1 : 23.74 secondes


2.	Affichage des informations basiques du jeu de données : quantité de lignes, colonnes, types des données, etc.

In [29]:
ex = 2

start_time = time.time()

print(f"Quantité de lignes : {data.height}")

print(data.head())

print(data.describe())

print(data.dtypes)

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

Quantité de lignes : 109950743
shape: (5, 9)
┌────────────┬────────────┬───────────┬───────────┬───┬──────────┬─────────┬───────────┬───────────┐
│ event_time ┆ event_type ┆ product_i ┆ category_ ┆ … ┆ brand    ┆ price   ┆ user_id   ┆ user_sess │
│ ---        ┆ ---        ┆ d         ┆ id        ┆   ┆ ---      ┆ ---     ┆ ---       ┆ ion       │
│ str        ┆ str        ┆ ---       ┆ ---       ┆   ┆ str      ┆ f64     ┆ i64       ┆ ---       │
│            ┆            ┆ i64       ┆ i64       ┆   ┆          ┆         ┆           ┆ str       │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪══════════╪═════════╪═══════════╪═══════════╡
│ 2019-10-01 ┆ view       ┆ 44600062  ┆ 210380745 ┆ … ┆ shiseido ┆ 35.79   ┆ 541312140 ┆ 72d76fde- │
│ 00:00:00   ┆            ┆           ┆ 959538772 ┆   ┆          ┆         ┆           ┆ 8bb3-4e00 │
│ UTC        ┆            ┆           ┆ 4         ┆   ┆          ┆         ┆           ┆ -8c23-a03 │
│            ┆            ┆           ┆       

3.	Affichage de la quantité de produits, marques et utilisateurs dans le dataset, sans répétitions

In [30]:
ex = 3

start_time = time.time()

print(f"Quantité de produits uniques dans le dataset :")
print(data.select(['product_id']).unique().height)
print(f"Quantité de marques uniques dans le dataset :")
print(data.select(['brand']).unique().height)
print(f"Quantité d'utilisateurs uniques dans le dataset :")
print(data.select(['user_id']).unique().height)

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

Quantité de produits uniques dans le dataset :
206876
Quantité de marques uniques dans le dataset :
4304
Quantité d'utilisateurs uniques dans le dataset :
5316649
Temps d'exécution de l'exercise 3 : 3.38 secondes


4. Affichage du top 5 de marques et catégories avec le prix moyen le plus élevé

In [32]:
ex = 4

start_time = time.time()

result_brand = data.group_by('brand').agg(avg_price = pl.col('price').mean().round(2))
result_brand = result_brand.sort(by='avg_price', descending=True)
print(result_brand.head())

result_category = data.group_by('category_id').agg(avg_price = pl.col('price').mean().round(2))
result_category = result_category.sort(by='avg_price', descending=True)
print(result_category.head())

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (5, 2)
┌─────────────┬───────────┐
│ brand       ┆ avg_price │
│ ---         ┆ ---       │
│ str         ┆ f64       │
╞═════════════╪═══════════╡
│ pinarello   ┆ 2533.24   │
│ nordictrack ┆ 2522.59   │
│ kessler     ┆ 2372.0    │
│ yjfitness   ┆ 2344.98   │
│ climadiff   ┆ 2261.95   │
└─────────────┴───────────┘
shape: (5, 2)
┌─────────────────────┬───────────┐
│ category_id         ┆ avg_price │
│ ---                 ┆ ---       │
│ i64                 ┆ f64       │
╞═════════════════════╪═══════════╡
│ 2104564977263182827 ┆ 1673.14   │
│ 2059484601444729123 ┆ 1666.93   │
│ 2053013564523807647 ┆ 1527.04   │
│ 2144356630513320518 ┆ 1489.79   │
│ 2179887855134835524 ┆ 1381.51   │
└─────────────────────┴───────────┘
Temps d'exécution de l'exercise 4 : 3.07 secondes


5.	Calcul du min, max et moyenne de prix par marque et par catégorie

In [35]:
ex = 5

start_time = time.time()

result_brand = data.group_by('brand').agg(
    min_price = pl.col('price').min().round(2),
    max_price = pl.col('price').max().round(2),
    avg_price = pl.col('price').mean().round(2)
)
print(result_brand.head())

result_category = data.group_by('category_id').agg(
    min_price = pl.col('price').min().round(2),
    max_price = pl.col('price').max().round(2),
    avg_price = pl.col('price').mean().round(2)
)
print(result_category.head())


end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (5, 4)
┌────────────┬───────────┬───────────┬───────────┐
│ brand      ┆ min_price ┆ max_price ┆ avg_price │
│ ---        ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪═══════════╪═══════════╪═══════════╡
│ trespass   ┆ 4.76      ┆ 4.76      ┆ 4.76      │
│ autovirazh ┆ 24.69     ┆ 125.09    ┆ 64.84     │
│ antares    ┆ 84.43     ┆ 89.06     ┆ 87.43     │
│ versal     ┆ 13.6      ┆ 17.6      ┆ 16.15     │
│ meccano    ┆ 28.32     ┆ 71.3      ┆ 58.76     │
└────────────┴───────────┴───────────┴───────────┘
shape: (5, 4)
┌─────────────────────┬───────────┬───────────┬───────────┐
│ category_id         ┆ min_price ┆ max_price ┆ avg_price │
│ ---                 ┆ ---       ┆ ---       ┆ ---       │
│ i64                 ┆ f64       ┆ f64       ┆ f64       │
╞═════════════════════╪═══════════╪═══════════╪═══════════╡
│ 2070747685681365440 ┆ 4.89      ┆ 4.89      ┆ 4.89      │
│ 2053013559440310913 ┆ 0.0       ┆ 977.89    ┆ 301

6.	Affichage des 5 prix les plus élevés

In [36]:
ex = 6

start_time = time.time()

result_price = data.select(['price']).unique().sort(by='price', descending=True)
print(result_price.head())

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (5, 1)
┌─────────┐
│ price   │
│ ---     │
│ f64     │
╞═════════╡
│ 2574.07 │
│ 2574.04 │
│ 2573.99 │
│ 2573.81 │
│ 2573.79 │
└─────────┘
Temps d'exécution de l'exercise 6 : 0.27 secondes


7.	Affichage de la quantité d’articles par marque et par catégorie, en utilisant une fonction d’agrégation

In [37]:
ex = 7

start_time = time.time()

result_brand = data.group_by('brand').agg(qty_articles = pl.col('product_id').count())
print(result_brand.head())

result_category = data.group_by('category_id').agg(qty_articles = pl.col('product_id').count())
print(result_category.head())

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (5, 2)
┌────────────────┬──────────────┐
│ brand          ┆ qty_articles │
│ ---            ┆ ---          │
│ str            ┆ u32          │
╞════════════════╪══════════════╡
│ babor          ┆ 8269         │
│ loreal         ┆ 1924         │
│ francescodonni ┆ 232          │
│ blaumann       ┆ 272          │
│ divalux        ┆ 2023         │
└────────────────┴──────────────┘
shape: (5, 2)
┌─────────────────────┬──────────────┐
│ category_id         ┆ qty_articles │
│ ---                 ┆ ---          │
│ i64                 ┆ u32          │
╞═════════════════════╪══════════════╡
│ 2053013558945383017 ┆ 132729       │
│ 2053013566209917945 ┆ 235644       │
│ 2053013556487520725 ┆ 162057       │
│ 2166064779414732801 ┆ 84           │
│ 2145603402644587402 ┆ 7057         │
└─────────────────────┴──────────────┘
Temps d'exécution de l'exercise 7 : 2.42 secondes


  8. Affichage de la quantité d’événements par type, en utilisant une fonction d’agrégation

In [38]:
ex = 8

start_time = time.time()

result = data.group_by('event_type').agg(qty_events = pl.col('event_time').count())
print(result.head())

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (3, 2)
┌────────────┬────────────┐
│ event_type ┆ qty_events │
│ ---        ┆ ---        │
│ str        ┆ u32        │
╞════════════╪════════════╡
│ purchase   ┆ 1659788    │
│ view       ┆ 104335509  │
│ cart       ┆ 3955446    │
└────────────┴────────────┘
Temps d'exécution de l'exercise 8 : 2.34 secondes


9.	Création et affichage d'un tableau qui contient la quantité d’événements de chaque type par marque

In [39]:
ex = 9

start_time = time.time()

result = data.group_by(['event_type', 'brand']).agg(qty_events = pl.col('event_time').count())
print(result.head())

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (5, 3)
┌────────────┬────────────┬────────────┐
│ event_type ┆ brand      ┆ qty_events │
│ ---        ┆ ---        ┆ ---        │
│ str        ┆ str        ┆ u32        │
╞════════════╪════════════╪════════════╡
│ view       ┆ electrolux ┆ 184688     │
│ view       ┆ hansa      ┆ 263334     │
│ view       ┆ intex      ┆ 84093      │
│ view       ┆ erlit      ┆ 56155      │
│ view       ┆ carters    ┆ 15952      │
└────────────┴────────────┴────────────┘
Temps d'exécution de l'exercise 9 : 1.62 secondes


10. Création et affichage d'un tableau qui contient la quantité d’événements de chaque type par marque et utilisateur

In [40]:
ex = 10

start_time = time.time()

result = data.group_by(['event_type', 'brand', 'user_id']).agg(qty_events = pl.col('event_time').count())
print(result.head())

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (5, 4)
┌────────────┬─────────┬───────────┬────────────┐
│ event_type ┆ brand   ┆ user_id   ┆ qty_events │
│ ---        ┆ ---     ┆ ---       ┆ ---        │
│ str        ┆ str     ┆ i64       ┆ u32        │
╞════════════╪═════════╪═══════════╪════════════╡
│ view       ┆ hp      ┆ 550050854 ┆ 3          │
│ view       ┆ respect ┆ 555447570 ┆ 1          │
│ view       ┆ vitek   ┆ 523239174 ┆ 14         │
│ view       ┆ null    ┆ 522832272 ┆ 38         │
│ view       ┆ samsung ┆ 518060648 ┆ 18         │
└────────────┴─────────┴───────────┴────────────┘
Temps d'exécution de l'exercise 10 : 5.03 secondes


11. Affichage du percentage de données vides ou nulles par colonne

In [41]:
ex = 11

start_time = time.time()

print(data.null_count() * 100 / data.height)

end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

shape: (1, 9)
┌────────────┬────────────┬────────────┬────────────┬───┬───────────┬───────┬─────────┬────────────┐
│ event_time ┆ event_type ┆ product_id ┆ category_i ┆ … ┆ brand     ┆ price ┆ user_id ┆ user_sessi │
│ ---        ┆ ---        ┆ ---        ┆ d          ┆   ┆ ---       ┆ ---   ┆ ---     ┆ on         │
│ f64        ┆ f64        ┆ f64        ┆ ---        ┆   ┆ f64       ┆ f64   ┆ f64     ┆ ---        │
│            ┆            ┆            ┆ f64        ┆   ┆           ┆       ┆         ┆ f64        │
╞════════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════╪═════════╪════════════╡
│ 0.0        ┆ 0.0        ┆ 0.0        ┆ 0.0        ┆ … ┆ 13.943738 ┆ 0.0   ┆ 0.0     ┆ 0.000011   │
└────────────┴────────────┴────────────┴────────────┴───┴───────────┴───────┴─────────┴────────────┘
Temps d'exécution de l'exercise 11 : 0.03 secondes


12. Suppression de lignes vides si le total par colonne ne dépasse pâs 15%

In [42]:
ex = 12

start_time = time.time()

total = data.height
print(f"Total de lignes : {total}")

col_to_del = []

for col in data.columns:
    is_na = (data.select(pl.col(col)).null_count()*100/total).item()
    if 0<is_na<15:
        col_to_del.append(col)

filtered_data = data.drop_nulls(subset=col_to_del)

total = filtered_data.height
print(f"Total de lignes : {total}")
    
end_time = time.time()

execution_time = round(end_time - start_time, 2)

timing.loc[ex, 'Timing_Polars'] = execution_time

print(f"Temps d'exécution de l'exercise {ex} : {execution_time} secondes")

Total de lignes : 109950743
Total de lignes : 94619490
Temps d'exécution de l'exercise 12 : 10.17 secondes


In [64]:
timing

,Timing_Pandas,Timing_Polars
1,76.65,23.74
2,9.76,3.42
3,6.35,3.38
4,5.59,3.07
5,9.90,3.68
6,0.86,0.27
7,5.25,2.42
8,4.34,2.34
9,10.54,1.62
10,26.66,5.03


In [65]:
timing.to_csv(timing_path)